In [17]:
import pandas as pd
import plotly.express as px
import requests

In [5]:
"""
This is called "filtered_projects" because I took the original china-seaport-dataset and removed
anything that's not Africa or South America. I could have done this purely in the dataframe but
to me this was easier.
"""
data = pd.read_csv("../data/china-seaport-dataset-filtered-china-south-america.csv")

data.sample(5)

,Unnamed: 0,port_name,recipient,region,commitment_year,aiddata_tuff_project_id,recommended_for_aggregates,umbrella,financier_country,implementation_start_year,...,funding_agencies,cofinancing_agencies,receiving_agencies,implementing_agencies,accountable_agencies,amount_constant_usd2021,latitude,longitude,osm_link,precision_level
3,3,Autonomous Port of Kribi,Cameroon,Africa,2017,53116,Yes,No,China (People's Republic of),0,...,Export-Import Bank of China (China Eximbank),NaN,"Cameroon Ministry of Economy, Planning, and Re...","China Harbour Engineering Co., Ltd.|Government...",NaN,1.705286e+08,2.728108,9.867072,https://www.openstreetmap.org/way/325526034,precise
31,77,Port of Cabinda,Angola,Africa,2016,66866,Yes,No,China (People's Republic of),0,...,China Development Bank (CDB),NaN,Government of Angola,China Gezhouba Group Company Ltd. (CGGC)|Gover...,NaN,6.612505e+07,-5.551103,12.195669,https://www.openstreetmap.org/way/234796859,precise
49,109,Rio Grande Container Terminal,Brazil,South America,2009,58633,No,No,China (People's Republic of),0,...,Export-Import Bank of China,NaN,Wilson Sons Limited,"Shanghai Zhenhua Heavy Industries Co., Ltd",NaN,2.508357e+07,-32.125991,-52.106431,https://www.openstreetmap.org/way/317560139,precise
51,113,Sierra Leone Fish Harbour,Sierra Leone,Africa,2019,62533,Yes,No,China (People's Republic of),0,...,China Ministry of Commerce,NaN,Government of Sierra Leone|Sierra Leone Minist...,Zhejiang Communications Construction Group Co Ltd,NaN,7.362901e+07,8.272123,-13.167560,NaN,approximate location
19,54,Lekki Deep Sea Port,Nigeria,Africa,2019,85628,Yes,No,China (People's Republic of),2020,...,China Development Bank (CDB),NaN,Lekki Port LFTZ Enterprise Limited,"China Harbour Engineering Co., Ltd.|Lekki Port...",China Export & Credit Insurance Corporation (S...,7.067221e+08,6.427862,4.009314,https://www.openstreetmap.org/way/785409163#ma...,precise


In [16]:
"""
We're going to try to visualize the amount of investment per country. This dataset
has duplicate entries (basically additional investments in the same port, etc). So we're 
going to combine the duplictates and sum their totals. Then we'll plot those totals.

UPDATE: Holy cow this was cursed. I can't believe I blew a whole day on just this cell.
"""
investment_data = data.groupby('recipient').agg(
    {'amount_constant_usd2021': 'sum', 'latitude': 'first', 'longitude': 'first'}).reset_index()

# This changes is for normalizing the dollar amount. '300M' instead of 300,000,000 or 
# 1.2B instead of 1,200,000,000.
def format_amount(amount):
    if amount >= 1e9:
        return f"{amount / 1e9:.1f}B"
    elif amount >= 1e6:
        return f"{amount / 1e6:.1f}M"
    else:
        return f"{amount:.0f}"

# This takes the above function and sets the formatted output as a column that we can then use to set the
# circle size.
investment_data['amount_formatted'] = investment_data['amount_constant_usd2021'].apply(
    format_amount)


"""
I wanted to put a circle over each country and have that circle be relative the amount invested.
The problem is the scaling between a couple hundred million and a billion was just too much. The 
billion dollar projects were eating the map. So now I'm categorizing the circle radius and capping it.
"""
def get_circle_size(amount):
    if amount > 1e9:
        return 50
    elif amount > 250e6:
        return 30
    elif amount > 100e6:
        return 20
    else:
        return 10

# This is lazy but now we're just setting the circle size as a column lol
investment_data['circle_size'] = investment_data['amount_constant_usd2021'].apply(
    get_circle_size)

# Using the plotly stuff from my midterm.
fig = px.scatter_geo(investment_data,
                     lat='latitude',
                     lon='longitude',
                     text='amount_formatted',
                     size='circle_size',
                     hover_name='recipient',
                     size_max=50,
                     title='Chinese Seaport Investments in Africa')

fig.update_geos(
    projection_type="natural earth",
    showcoastlines=True,
    showcountries=True,
    scope="africa"
)

fig.update_layout(
    geo=dict(
        showcoastlines=True,
        showcountries=True
    ),
    width=1200,
    height=800
)

fig.show()

In [18]:
"""
The next step is to grab the GDP and total debt for each country. China does its long-term planning on low income
countries in order to 'debt trap' them with loans China knows they can't pay off in the long term. We'll turn these 
figures into a ratio which we will use as a predictor for the model later.
"""

# Filter out duplicates based on the recipient column
unique_recipients = data['recipient'].drop_duplicates().reset_index(drop=True)

# Define the indicators for GDP and total debt
gdp_indicator = 'NY.GDP.MKTP.CD'
debt_indicator = 'GC.DOD.TOTL.GD.ZS'

# Initialize lists to store GDP and debt data
gdp_list = []
debt_list = []

# Fetch GDP and total debt data for each unique recipient
for country in unique_recipients:
    # Fetch GDP data
    gdp_url = f'http://api.worldbank.org/v2/country/{
        country}/indicator/{gdp_indicator}?format=json&per_page=1'
    gdp_response = requests.get(gdp_url)
    if gdp_response.status_code == 200 and len(gdp_response.json()) > 1:
        gdp_value = gdp_response.json()[1][0]['value']
    else:
        gdp_value = None
    gdp_list.append(gdp_value)

    # Fetch total debt data
    debt_url = f'http://api.worldbank.org/v2/country/{
        country}/indicator/{debt_indicator}?format=json&per_page=1'
    debt_response = requests.get(debt_url)
    if debt_response.status_code == 200 and len(debt_response.json()) > 1:
        debt_value = debt_response.json()[1][0]['value']
    else:
        debt_value = None
    debt_list.append(debt_value)

# Create a new dataframe with the fetched data
gdp_debt_df = pd.DataFrame({
    'recipient': unique_recipients,
    'gdp': gdp_list,
    'debt': debt_list
})

# Merge the new dataframe with the original data
merged_data = pd.merge(data, gdp_debt_df, on='recipient', how='left')

# Display the merged dataframe
merged_data.head()

,Unnamed: 0,port_name,recipient,region,commitment_year,aiddata_tuff_project_id,recommended_for_aggregates,umbrella,financier_country,implementation_start_year,...,receiving_agencies,implementing_agencies,accountable_agencies,amount_constant_usd2021,latitude,longitude,osm_link,precision_level,gdp,debt
0,0,Alto do Bandim Fishing Port,Guinea-Bissau,Africa,2017,57221,Yes,No,China (People's Republic of),2018,...,Government of Guinea-Bissau,China Railway Guangzhou Engineering Bureau Gro...,NaN,2.711405e+07,11.839010,-15.588719,https://www.openstreetmap.org/node/5648640456,boundary not clear,None,None
1,1,Autonomous Port of Abidjan,Cote d'Ivoire,Africa,2014,30857,Yes,No,China (People's Republic of),2015,...,Abidjan Autonomous Port Authority (PAA)|Govern...,Abidjan Autonomous Port Authority (PAA)|Bollor...,NaN,8.783583e+08,5.256896,-4.011836,https://www.openstreetmap.org/way/852907664,boundary not clear,None,None
2,2,Autonomous Port of Kribi,Cameroon,Africa,2011,350,Yes,No,China (People's Republic of),2010,...,"Cameroon Ministry of Economy, Planning, and Re...","Cameroon Ministry of Economy, Planning, and Re...",NaN,5.202393e+08,2.728108,9.867072,https://www.openstreetmap.org/way/325526034,precise,None,None
3,3,Autonomous Port of Kribi,Cameroon,Africa,2017,53116,Yes,No,China (People's Republic of),0,...,"Cameroon Ministry of Economy, Planning, and Re...","China Harbour Engineering Co., Ltd.|Government...",NaN,1.705286e+08,2.728108,9.867072,https://www.openstreetmap.org/way/325526034,precise,None,None
4,4,Autonomous Port of Kribi,Cameroon,Africa,2017,53396,Yes,No,China (People's Republic of),0,...,"Cameroon Ministry of Economy, Planning, and Re...","China Harbour Engineering Co., Ltd.|Government...",NaN,6.044935e+08,2.728108,9.867072,https://www.openstreetmap.org/way/325526034,precise,None,None


In [20]:
merged_data.sample(10)

,Unnamed: 0,port_name,recipient,region,commitment_year,aiddata_tuff_project_id,recommended_for_aggregates,umbrella,financier_country,implementation_start_year,...,receiving_agencies,implementing_agencies,accountable_agencies,amount_constant_usd2021,latitude,longitude,osm_link,precision_level,gdp,debt
54,119,Port Sudan,Sudan,Africa,2001,97783,Yes,No,China (People's Republic of),2001,...,Sea Ports Corporation (SPC),"China Harbour Engineering Co., Ltd. (CHEC)",NaN,2.778564e+07,19.598400,37.227000,https://www.openstreetmap.org/way/627215576,precise,None,None
45,104,Porto Sul Intermodal Logistics Complex,Brazil,South America,2017,62855,No,No,China (People's Republic of),0,...,Government of Brazil,Bahia Mineração (BAMIN)|China Communications C...,NaN,0.000000e+00,-14.745549,-39.195522,NaN,approximate location,None,None
41,100,Port of Tema,Ghana,Africa,2016,59302,Yes,No,China (People's Republic of),2016,...,Meridian Port Services Limited,"AECOM|China Harbour Engineering Co., Ltd.",NaN,2.727658e+08,5.615523,-0.014969,https://www.openstreetmap.org/way/666974334,precise,None,None
18,50,Kisumu Port,Kenya,Africa,2017,52927,No,No,China (People's Republic of),0,...,Government of Kenya,Kenya Ports Authority (KPA),NaN,1.613273e+08,-0.101879,34.746292,https://www.openstreetmap.org/way/499869532,precise,None,None
37,87,Port of Moroni,Comoros,Africa,2018,57664,No,No,China (People's Republic of),0,...,Government of Comoros,China Road & Bridge Corporation (CRBC),NaN,1.700523e+08,-11.705466,43.247852,https://www.openstreetmap.org/way/144754155,precise,None,None
40,97,Port of Takoradi,Ghana,Africa,2011,73196,No,No,China (People's Republic of),0,...,Government of Ghana,"China Harbour Engineering Co., Ltd.",NaN,0.000000e+00,4.887073,-1.735196,https://www.openstreetmap.org/way/756865759,boundary not clear,None,None
33,79,Port of Caio,Angola,Africa,2016,68708,Yes,No,China (People's Republic of),0,...,Government of Angola,"Angola Ministry of Transportation|Caioporto, S...",China Export & Credit Insurance Corporation (S...,1.100510e+09,-5.478222,12.226776,https://www.openstreetmap.org/way/977011440#ma...,precise,None,None
34,81,Port of Dar es Salaam,Tanzania,Africa,2012,23231,No,No,China (People's Republic of),0,...,Government of Tanzania,"China Communications Construction Co., Ltd. (C...",NaN,6.896215e+08,-6.835618,39.292597,https://www.openstreetmap.org/way/277786153#ma...,boundary not clear,None,None
14,22,DP World Posorja,Ecuador,South America,2018,73745,Yes,No,China (People's Republic of),0,...,DP Word Posorja,"China Harbour Engineering Co., Ltd.|Guayaquil ...",NaN,5.449249e+07,-2.693399,-80.248845,NaN,approximate location,None,None
28,74,Port of Banjul,Gambia,Africa,2017,56668,No,No,China (People's Republic of),0,...,Government of Gambia,"China Communications Construction Co., Ltd. (C...",NaN,2.040536e+08,13.444198,-16.575750,https://www.openstreetmap.org/relation/3694538,precise,None,None
